In [1]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
mse = torch.nn.MSELoss()

# Split Data

# Train Model

In [3]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [4]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))

Files already downloaded
3000
750


In [5]:
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [6]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.latent_vars = 10
        self.fc11 = nn.Linear(784, 400)
        self.fc12 = nn.Linear(400, 100)
        self.fc21 = nn.Linear(100, 20)

        self.fc3 = nn.Linear(10, 400)
        self.fc4 = nn.Linear(400, 784)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        
        h1 = self.relu(self.fc11(x))
        h2 = self.relu(self.fc12(h1))
        
        z = self.fc21(h2)
        mu = z[:, 0:self.latent_vars]
        log_sig = z[:, self.latent_vars:]
        return mu, log_sig

    def reparametrize(self, mu, logvar):
        
        eps = Variable(torch.randn(logvar.size()))

        z = mu + torch.exp(logvar / 2) * eps

        return z

    def decode(self, z):
        
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar




In [233]:
### Adding convolution layers to VAE

class ConvVAE(nn.Module):
    def __init__(self):
        super(ConvVAE, self).__init__()
        self.latent_vars = 10
        
        self.fc11 = nn.Linear(64, 20)
        self.fc12 = nn.Linear(128, 64)
        
        
        self.conv1 = nn.Conv2d(1, 32, 3, 3, 1, 1, 1, 1)
        self.conv2 = nn.Conv2d(32, 16, 3, 3, 1, 1, 1, 1)
        self.conv3 = nn.Conv2d(16, 16, 3, 3, 1, 1, 1, 1)
        
    
        
        self.dconv1 = nn.ConvTranspose2d(16, 16, 3,3,2,2)
        self.dconv2 = nn.ConvTranspose2d(16, 32, 3,3,2,2)
        self.dconv3 = nn.ConvTranspose2d(32, 1, 3,3,2,2)
        
        self.dconv11 = nn.Conv2d(16, 16, kernel_size= 3)
        self.dconv21 = nn.Conv2d(16, 32, kernel_size= 3)
        self.dconv31 = nn.Conv2d(32, 1, kernel_size= 3)
        
        self.flat_dim = self.get_flat_dim()

        self.h = self.flat_dim[0] *self.flat_dim[1] *self.flat_dim[2]
        print(self.flat_dim)
        self.bn1 = nn.BatchNorm2d(128)
        self.bn2 = nn.BatchNorm2d(64)

        self.fc3 = nn.Linear(10, self.h)
        #self.fc4 = nn.Linear(400, 784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def get_flat_dim(self):

        x = Variable(torch.randn(64,1,28,28))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        return(x.size()[1:])

    def encode(self, x):
        
        x = self.relu(self.conv1(x))
        #x = F.max_pool2d(x, 2)
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        #print("Size after encoding", x.size())
        x = x.view(-1, 16*2*2)
        #print(x.size())
        z = self.fc11(x)
        mu = z[:, 0:self.latent_vars]
        log_sig = z[:, self.latent_vars:]
      
        #print(mu.size(), log_sig.size())
        return mu, log_sig

    def reparametrize(self, mu, logvar):
        eps = Variable(torch.randn(logvar.size()))
        z = mu + torch.exp(logvar / 2) * eps
        return z

    def decode(self, x):
        #print("Input size 0", x.size())
        x = F.relu(self.fc3(x))
        #print("Checkpoint 0", x.size())
        x = x.view(-1,16, 2,2)
        #print("Checkpoint 1", x.size())
        x = self.dconv1(x)
        #print("Checkpoint 2", x.size())
        x = self.dconv2(x)
        #print("Checkpoint 3", x.size())
        x = self.dconv3(x)
        #print(x.size())
        
        return x #self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar



(16, 2, 2)


In [ ]:
### Adding DenoisingAE

class DenoiseAE(nn.Module):
    def __init__(self):
        super(ConvVAE, self).__init__()
        self.latent_vars = 10
        
        self.fc11 = nn.Linear(64, 20)
        self.fc12 = nn.Linear(128, 64)
        
        
        self.conv1 = nn.Conv2d(1, 32, 3, 3, 1, 1, 1, 1)
        self.conv2 = nn.Conv2d(32, 16, 3, 3, 1, 1, 1, 1)
        self.conv3 = nn.Conv2d(16, 16, 3, 3, 1, 1, 1, 1)
        
    
        
        self.dconv1 = nn.ConvTranspose2d(16, 16, 3,3,2,2)
        self.dconv2 = nn.ConvTranspose2d(16, 32, 3,3,2,2)
        self.dconv3 = nn.ConvTranspose2d(32, 1, 3,3,2,2)
        
        self.dconv11 = nn.Conv2d(16, 16, kernel_size= 3)
        self.dconv21 = nn.Conv2d(16, 32, kernel_size= 3)
        self.dconv31 = nn.Conv2d(32, 1, kernel_size= 3)
        
        self.flat_dim = self.get_flat_dim()

        self.h = self.flat_dim[0] *self.flat_dim[1] *self.flat_dim[2]
        print(self.flat_dim)
        self.bn1 = nn.BatchNorm2d(128)
        self.bn2 = nn.BatchNorm2d(64)

        self.fc3 = nn.Linear(10, self.h)
        #self.fc4 = nn.Linear(400, 784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def get_flat_dim(self):

        x = Variable(torch.randn(64,1,28,28))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        return(x.size()[1:])

    def encode(self, x):
        
        x = self.relu(self.conv1(x))
        #x = F.max_pool2d(x, 2)
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        #print("Size after encoding", x.size())
        x = x.view(-1, 16*2*2)
        #print(x.size())
        z = self.fc11(x)
        mu = z[:, 0:self.latent_vars]
        log_sig = z[:, self.latent_vars:]
      
        #print(mu.size(), log_sig.size())
        return mu, log_sig

    def reparametrize(self, mu, logvar):
        eps = Variable(torch.randn(logvar.size()))
        z = mu + torch.exp(logvar / 2) * eps
        return z

    def decode(self, x):
        #print("Input size 0", x.size())
        x = F.relu(self.fc3(x))
        #print("Checkpoint 0", x.size())
        x = x.view(-1,16, 2,2)
        #print("Checkpoint 1", x.size())
        x = self.dconv1(x)
        #print("Checkpoint 2", x.size())
        x = self.dconv2(x)
        #print("Checkpoint 3", x.size())
        x = self.dconv3(x)
        #print(x.size())
        
        return x #self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [234]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False


def loss_function(recon_x, x, mu, logvar):
    mean_squared =mse(X_hat, data)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return mean_squared + KLD






In [243]:
model = ConvVAE()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

(16, 2, 2)


In [244]:
# CPU only training
def generate_train(epoch):
    model.train()
    train_loss = 0
    classification_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        x_new, mu, logvar = model(data)
        kl_loss = 0.5 * torch.mean(torch.exp(logvar) + mu ** 2 - 1. - logvar)
        recon_loss = mse(x_new, data) + kl_loss
        output = F.log_softmax(mu)
        class_loss = F.nll_loss(output, target)
        loss = recon_loss + class_loss
        loss.backward()

        train_loss += loss.data[0]
        classification_loss += class_loss.data[0]
        optimizer.step()
        #if batch_idx % 100 == 0:
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
             #   epoch, batch_idx * len(data), len(train_loader.dataset),
            #  100. * batch_idx / len(train_loader),
        #    loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, classification_loss / len(train_loader.dataset)))

    
def generate_test(epoch, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        x_new, mu, logvar = model(data)
        output = F.log_softmax(mu) 
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




In [ ]:
#Generative phase
for epoch in range(1, 50):
    generate_train(epoch)
    generate_test(epoch, valid_loader)

====> Epoch: 1 Average loss: 0.0360

Test set: Average loss: 0.0360, Accuracy: 1622/10000 (16%)

====> Epoch: 2 Average loss: 0.0358

Test set: Average loss: 0.0358, Accuracy: 2321/10000 (23%)

====> Epoch: 3 Average loss: 0.0355

Test set: Average loss: 0.0354, Accuracy: 2959/10000 (30%)

====> Epoch: 4 Average loss: 0.0350

Test set: Average loss: 0.0347, Accuracy: 3624/10000 (36%)

====> Epoch: 5 Average loss: 0.0339

Test set: Average loss: 0.0333, Accuracy: 5165/10000 (52%)

====> Epoch: 6 Average loss: 0.0321

Test set: Average loss: 0.0311, Accuracy: 6017/10000 (60%)

====> Epoch: 7 Average loss: 0.0295

Test set: Average loss: 0.0283, Accuracy: 6328/10000 (63%)

====> Epoch: 8 Average loss: 0.0265

Test set: Average loss: 0.0254, Accuracy: 6778/10000 (68%)

====> Epoch: 9 Average loss: 0.0239

Test set: Average loss: 0.0232, Accuracy: 6964/10000 (70%)

====> Epoch: 10 Average loss: 0.0220

Test set: Average loss: 0.0218, Accuracy: 7107/10000 (71%)

====> Epoch: 11 Average loss:

In [13]:
# Check generated images
from  torchvision.utils import save_image
for index, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    gen_data, mu, logvar = model(data)
    gen_images = gen_data.view(-1, 1,28,28)
    #print(data[0])
    save_image(gen_images.data, 'images/gen{}.jpg'.format(index))


In [ ]:
# CPU only training
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.8)
def train(epoch):
    classifier.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        gen_data, mu, logvar = model(data)
        #Check reconstruction loss 
        recon_loss = reconstruction_function(gen_data, data)
        #print("Reconstruction loss is", recon_loss)
        #print("Type of generated data", type(gen_data))
        
        optimizer.zero_grad()
        output = classifier(gen_data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    classifier.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        gen_data, mu, logvar = model(data)
        output = classifier(gen_data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))
    


In [ ]:
#Classification phase
for epoch in range(1, 10):
    train(epoch)
    test(epoch, valid_loader)

# Create Sample Submission

In [ ]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

In [ ]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [ ]:
test(1, test_loader)

In [ ]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [ ]:
label_predict

In [ ]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [ ]:
predict_label.head()

In [ ]:
predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)
